## Convolutional Neural Network Template

Classification ( / Regression) CNN Template

In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "" # If you want to use TF CPU version while having the GPU version installed

import tensorflow as tf
from tensorflow.keras import Sequential, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import plotly.offline as ply
import plotly.graph_objs as graphs
import math
import random
import matplotlib.pyplot as plt

2022-09-06 17:41:28.826306: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Seeding random number generators to obtain reproducible results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value) # Resets itself on every use
tf.random.set_seed(seed_value) # tf.set_random_seed(seed_value) on older TF versions

In [3]:
def load_data(file_path):
    # Load CSV file to pandas data frame
    df = pd.read_csv(file_path, header=0)

    # Split data frame columns into features and labels
    y = df["label"].values
    X_flat = df.drop("label", axis=1).values
    X = [np.resize(array, (28, 28)) for array in X_flat] # Resize flat vector to 28x28
    X = np.array([np.reshape(sample, (sample.shape[0], sample.shape[1], 1)) for sample in X]) # Add color channel
    
    return X, tf.keras.utils.to_categorical(y) # Labels are returned as one-hot encoded vectors

In [4]:
def plot_train_val_performances(performances, metric="Accuracy", show=True):
    """Plot training vs. testing accuracy over all epochs. performances is a dictionary mapping epoch numbers
       as integers to lists containing training and validation performance (e.g. accuracy).
       metric is a string indicating the used performance metric."""
    x = list(performances.keys())     # Number of epochs
    y_train = [i[0] for i in performances.values()]
    y_val = [i[1] for i in performances.values()]

    trace_train = graphs.Scatter(x=x, y=y_train, name="Training", mode="lines+markers",
                                 line=dict(width=4),
                                 marker=dict(symbol="circle",
                                             size=10))
    trace_val = graphs.Scatter(x=x, y=y_val, name="Validation", mode="lines+markers",
                                line=dict(width=4),
                                marker=dict(symbol="circle",
                                            size=10))

    layout = graphs.Layout(title="Training vs. Validation {}".format(metric),
                           xaxis={"title": "Epoch"},
                           yaxis={"title": metric})

    fig = graphs.Figure(data=[trace_train, trace_val], layout=layout)
    ply.plot(fig, filename="plotly_train_val_{}.html".format(metric), auto_open=show)
    print("Plot saved as plotly_train_val_{}.html".format(metric))

In [5]:
def create_2DCNN_model(input_shape):
    """Build architecture of the model"""
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape,
                            activation="relu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="selu"))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation="selu"))
    model.add(layers.Dense(10, activation="softmax"))

    # Create model
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [6]:
# Specify data location
data_dir = r"Jupyter-notebooks_Convolutional_Neural_Networks/Data/mnist.csv"

# Set hyperparameters
num_epochs = 200
batch_size = 64
dims = (28, 28, 1)

In [7]:
# Load data
X, y = load_data(data_dir)

# Determine split sizes
train_size = math.floor(0.02 * len(y)) # ONLY 2 % of the data set is used for training
val_size = math.floor(0.475 * len(y))
test_size = math.floor(0.475 * len(y))

# Create splitted sets
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

In [8]:
# Create CNN model
model = create_2DCNN_model(dims)

2022-09-06 17:41:30.703251: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-06 17:41:30.704055: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-06 17:41:30.772995: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-09-06 17:41:30.773060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 17:41:30.773658: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:42:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.

In [9]:
# Data augmentation using Keras' built-in data generator
train_generator = ImageDataGenerator(rotation_range=15,
                                     zoom_range = 0.1,
                                     width_shift_range=2,
                                     height_shift_range=2)

In [10]:
# Configure stopping criterion via early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                            patience=10, 
                                            mode="min",
                                            restore_best_weights=True)

In [11]:
# Training model while dynamically retrieving the augmented data from the generator
train_summary = model.fit_generator(generator=train_generator.flow(X_train, y_train, batch_size=32),
                                    validation_data=(X_val, y_val),
                                    epochs=num_epochs,
                                    callbacks=[callback],
                                    verbose=1)

/home/cspielvogel/anaconda3/envs/tensorflow2/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.

2022-09-06 17:41:31.912623: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-06 17:41:31.913279: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3499610000 Hz


Epoch 1/200


2022-09-06 17:41:32.335746: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-09-06 17:41:32.496924: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-09-06 17:41:33.284325: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-09-06 17:41:33.327091: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


7/7 [==============================] - 16s 1s/step - loss: 9.5544 - accuracy: 0.1663 - val_loss: 2.8335 - val_accuracy: 0.1743
Epoch 2/200
7/7 [==============================] - 0s 47ms/step - loss: 3.0903 - accuracy: 0.1775 - val_loss: 2.1142 - val_accuracy: 0.3118
Epoch 3/200
7/7 [==============================] - 0s 48ms/step - loss: 1.9727 - accuracy: 0.3487 - val_loss: 1.4398 - val_accuracy: 0.5242
Epoch 4/200
7/7 [==============================] - 0s 46ms/step - loss: 1.6385 - accuracy: 0.4361 - val_loss: 1.3321 - val_accuracy: 0.6168
Epoch 5/200
7/7 [==============================] - 0s 47ms/step - loss: 1.1362 - accuracy: 0.6438 - val_loss: 0.9101 - val_accuracy: 0.6931
Epoch 6/200
7/7 [==============================] - 0s 47ms/step - loss: 0.9489 - accuracy: 0.7040 - val_loss: 0.7020 - val_accuracy: 0.7661
Epoch 7/200
7/7 [==============================] - 0s 45ms/step - loss: 0.7961 - accuracy: 0.7525 - val_loss: 0.6881 - val_accuracy: 0.7691
Epoch 8/200
7/7 [================

In [12]:
# Evaluate fitted model using test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("\nTest ACC:", round(test_acc, 3))

158/158 [==============================] - 3s 17ms/step - loss: 0.2219 - accuracy: 0.9319

Test ACC: 0.932


In [13]:
# Get epochwise performances
train_acc = train_summary.history["accuracy"]
val_acc = train_summary.history["val_accuracy"]

# Format and store performances per epoch for plotting
accs = {epoch: [round(performance[0], 2), round(performance[1], 2)]
        for epoch, performance in enumerate(zip(train_acc, val_acc))}

# Save model
model.save_weights("model_weights.h5")

# Plot training and validation performance over epochs
plot_train_val_performances(accs, "Accuracy")

Plot saved as plotly_train_val_Accuracy.html
